<a href="https://colab.research.google.com/github/2359181042/GNN_cs224w/blob/main/HAIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 5.3MB/s 
     |████████████████████████████████| 1.4MB 4.1MB/s 
     |████████████████████████████████| 225kB 5.3MB/s 
     |████████████████████████████████| 235kB 35.6MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import os
import json
import numpy as np
import pandas as pd
TIME_WINDOW = 24
PRED_TIME = 12
DATA_PATH = 'gdrive/My Drive/HAir/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/gdrive


In [4]:
class trainDataset(Data.Dataset):
    def __init__(self, transform=None, train=True):
        with open(os.path.join(DATA_PATH,'city_train.txt'), 'r') as f:
            self.cities = json.load(f)

        with open(os.path.join(DATA_PATH,'jiaxing_train.txt'), 'r') as f:
            self.jiaxing = json.load(f)

        with open(os.path.join(DATA_PATH,'shanghai_train.txt'), 'r') as f:
            self.shanghai = json.load(f)

        with open(os.path.join(DATA_PATH,'suzhou_train.txt'), 'r') as f:
            self.suzhou = json.load(f)

        with open(os.path.join(DATA_PATH,'stations.txt'), 'r') as f:
            self.stations = json.load(f)

    def GetCityData(self,city_name,city_source,index):
        station_list = self.stations[city_name]
        city_aqi = []
        city_y = []
        for x in station_list:
            city_aqi.append(city_source[x][index][:TIME_WINDOW])
            city_y.append(city_source[x][index][TIME_WINDOW:])

        city_aqi = torch.FloatTensor(city_aqi)
        city_y = torch.FloatTensor(city_y)
        city_sim = torch.FloatTensor(city_source['sim'][index])
        city_conn = torch.tensor(city_source['conn'])
        city_weather = torch.FloatTensor(city_source['weather'][index])
        city_for = torch.FloatTensor(city_source['weather_for'][index])
        city_poi = torch.FloatTensor(city_source['poi'])

        city_data = [city_aqi, city_conn, city_poi, city_sim,
                city_weather, city_for, city_y]
		
        return city_data


    def __getitem__(self, index):
        jiaxing_data = self.GetCityData('jiaxing',self.jiaxing,index)
        shanghai_data = self.GetCityData('shanghai',self.shanghai,index)
        suzhou_data = self.GetCityData('suzhou',self.suzhou,index)

        cities_aqi = torch.FloatTensor(self.cities['aqi'][index])
        cities_conn = torch.tensor(self.cities['conn'])
        cities_weather = torch.FloatTensor(self.cities['weather'][index])
        cities_sim = torch.FloatTensor(self.cities['sim'][index])

        cities_data = [cities_aqi, cities_conn,cities_sim,cities_weather]
        
        return cities_data,jiaxing_data,shanghai_data,suzhou_data

    def __len__(self):
        return len(self.shanghai['weather'])

class valDataset(Data.Dataset):
    def __init__(self, transform=None, train=True):
        with open(os.path.join(DATA_PATH,'city_val.txt'), 'r') as f:
            self.cities = json.load(f)

        with open(os.path.join(DATA_PATH,'jiaxing_val.txt'), 'r') as f:
            self.jiaxing = json.load(f)

        with open(os.path.join(DATA_PATH,'shanghai_val.txt'), 'r') as f:
            self.shanghai = json.load(f)

        with open(os.path.join(DATA_PATH,'suzhou_val.txt'), 'r') as f:
            self.suzhou = json.load(f)

        with open(os.path.join(DATA_PATH,'stations.txt'), 'r') as f:
            self.stations = json.load(f)

    def GetCityData(self,city_name,city_source,index):
        station_list = self.stations[city_name]
        city_aqi = []
        city_y = []
        for x in station_list:
            city_aqi.append(city_source[x][index][:TIME_WINDOW])
            city_y.append(city_source[x][index][TIME_WINDOW:])

        city_aqi = torch.FloatTensor(city_aqi)
        city_y = torch.FloatTensor(city_y)
        city_sim = torch.FloatTensor(city_source['sim'][index])
        city_conn = torch.tensor(city_source['conn'])
        city_weather = torch.FloatTensor(city_source['weather'][index])
        city_for = torch.FloatTensor(city_source['weather_for'][index])
        city_poi = torch.FloatTensor(city_source['poi'])

        city_data = [city_aqi, city_conn, city_poi, city_sim,
                city_weather, city_for, city_y]
		
        return city_data


    def __getitem__(self, index):
        jiaxing_data = self.GetCityData('jiaxing',self.jiaxing,index)
        shanghai_data = self.GetCityData('shanghai',self.shanghai,index)
        suzhou_data = self.GetCityData('suzhou',self.suzhou,index)

        cities_aqi = torch.FloatTensor(self.cities['aqi'][index])
        cities_conn = torch.tensor(self.cities['conn'])
        cities_weather = torch.FloatTensor(self.cities['weather'][index])
        cities_sim = torch.FloatTensor(self.cities['sim'][index])

        cities_data = [cities_aqi, cities_conn,cities_sim,cities_weather]
        
        return cities_data,jiaxing_data,shanghai_data,suzhou_data

    def __len__(self):
        return len(self.shanghai['weather'])


class testDataset(Data.Dataset):
    def __init__(self, transform=None, train=True):
        with open(os.path.join(DATA_PATH,'city_test.txt'), 'r') as f:
            self.cities = json.load(f)

        with open(os.path.join(DATA_PATH,'jiaxing_test.txt'), 'r') as f:
            self.jiaxing = json.load(f)

        with open(os.path.join(DATA_PATH,'shanghai_test.txt'), 'r') as f:
            self.shanghai = json.load(f)

        with open(os.path.join(DATA_PATH,'suzhou_test.txt'), 'r') as f:
            self.suzhou = json.load(f)

        with open(os.path.join(DATA_PATH,'stations.txt'), 'r') as f:
            self.stations = json.load(f)

    def GetCityData(self,city_name,city_source,index):
        station_list = self.stations[city_name]
        city_aqi = []
        city_y = []
        for x in station_list:
            city_aqi.append(city_source[x][index][:TIME_WINDOW])
            city_y.append(city_source[x][index][TIME_WINDOW:])

        city_aqi = torch.FloatTensor(city_aqi)
        city_y = torch.FloatTensor(city_y)
        city_sim = torch.FloatTensor(city_source['sim'][index])
        city_conn = torch.tensor(city_source['conn'])
        city_weather = torch.FloatTensor(city_source['weather'][index])
        city_for = torch.FloatTensor(city_source['weather_for'][index])
        city_poi = torch.FloatTensor(city_source['poi'])

        city_data = [city_aqi, city_conn, city_poi, city_sim,
                city_weather, city_for, city_y]
		
        return city_data


    def __getitem__(self, index):
        jiaxing_data = self.GetCityData('jiaxing',self.jiaxing,index)
        shanghai_data = self.GetCityData('shanghai',self.shanghai,index)
        suzhou_data = self.GetCityData('suzhou',self.suzhou,index)

        cities_aqi = torch.FloatTensor(self.cities['aqi'][index])
        cities_conn = torch.tensor(self.cities['conn'])
        cities_weather = torch.FloatTensor(self.cities['weather'][index])
        cities_sim = torch.FloatTensor(self.cities['sim'][index])

        cities_data = [cities_aqi, cities_conn,cities_sim,cities_weather]
        
        return cities_data,jiaxing_data,shanghai_data,suzhou_data

    def __len__(self):
        return len(self.shanghai['weather'])

In [5]:
import argparse
import json
import math
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data

#from dataset import testDataset, trainDataset, valDataset
#from model import CityModel, GlobalModel

from torch_geometric.nn import MetaLayer

In [14]:
parser = argparse.ArgumentParser(description='Multi-city AQI forecasting')
parser.add_argument('--device', type=str, default='cuda', help='')
parser.add_argument('--run_times', type=int, default=5, help='')
parser.add_argument('--epoch', type=int, default=300, help='')
parser.add_argument('--batch_size', type=int, default=128, help='')
parser.add_argument('--city_num', type=int, default=10, help='')
parser.add_argument('--gnn_h', type=int, default=32, help='')
parser.add_argument('--rnn_h', type=int, default=64, help='')
parser.add_argument('--rnn_l', type=int, default=1, help='')
parser.add_argument('--aqi_em', type=int, default=16, help='')
parser.add_argument('--poi_em', type=int, default=8, help='poi embedding')
parser.add_argument('--wea_em', type=int, default=12, help='wea embedding')
parser.add_argument('--lr', type=float, default=0.001, help='lr')
parser.add_argument('--wd', type=float, default=0.001, help='weight decay')
parser.add_argument('--pred_step', type=int, default=12, help='step')
args = parser.parse_args()
print('this is arges',args)
device = args.device

usage: ipykernel_launcher.py [-h] [--device DEVICE] [--run_times RUN_TIMES]
                             [--epoch EPOCH] [--batch_size BATCH_SIZE]
                             [--city_num CITY_NUM] [--gnn_h GNN_H]
                             [--rnn_h RNN_H] [--rnn_l RNN_L] [--aqi_em AQI_EM]
                             [--poi_em POI_EM] [--wea_em WEA_EM] [--lr LR]
                             [--wd WD] [--pred_step PRED_STEP]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-cbf40351-76c0-4104-9be8-1e1693757acb.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
train_dataset = trainDataset()
train_loader = Data.DataLoader(train_dataset,batch_size=128,shuffle=True)
val_dataset = valDataset()
val_loader = Data.DataLoader(val_dataset,
                             batch_size=128,
                             shuffle=True)

test_dataset = testDataset()
test_loader = Data.DataLoader(test_dataset,
                              batch_size=128,
                              shuffle=False)

## model part

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean, scatter_add


class RNNEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True)

    def forward(self, x, h0, c0):
        # Set initial hidden and cell states
        # Forward propagate LSTM
        out, (h_n, c_n) = self.lstm(x, (h0, c0))  
        # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        return h_n, c_n


class RNNDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True)
        self.lin = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()

    def forward(self, x, h_0, c_0):
        # Forward propagate LSTM
        out, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = self.lin(out)
        out = self.relu(out)
        # Decode the hidden state of the last time step
        return out, h_n, c_n


class GlobalModel(torch.nn.Module):
    def __init__(self, aqi_em, rnn_h, rnn_l, gnn_h):
        super(GlobalModel, self).__init__()
        self.aqi_em = aqi_em
        self.rnn_h = rnn_h
        self.gnn_h = gnn_h
        self.aqi_embed = Seq(Lin(1, aqi_em), ReLU())
        self.aqi_rnn = nn.LSTM(aqi_em, rnn_h, rnn_l, batch_first=True)
        self.city_gnn = CityGNN(rnn_h, 2, gnn_h)

    def batchInput(self, x, edge_w, edge_conn):
        sta_num = x.shape[1]
        x = x.reshape(-1, x.shape[-1])
        edge_w = edge_w.reshape(-1, edge_w.shape[-1])
        for i in range(edge_conn.size(0)):
            edge_conn[i, :] = torch.add(edge_conn[i, :], i * sta_num)
        edge_conn = edge_conn.transpose(0, 1)
        edge_conn = edge_conn.reshape(2, -1)
        return x, edge_w, edge_conn

    def forward(self, city_aqi, city_conn, city_w, city_num):
        city_aqi = city_aqi.unsqueeze(dim=-1)
        city_aqi = self.aqi_embed(city_aqi)
        city_aqi, _ = self.aqi_rnn(city_aqi.reshape(-1, 24, self.aqi_em))
        city_aqi = city_aqi.reshape(-1, 10, 24, self.rnn_h)
        city_aqi = city_aqi.transpose(1, 2)
        city_aqi = city_aqi.reshape(-1, city_num, city_aqi.shape[-1])

        city_conn = city_conn.transpose(1, 2).repeat(24, 1, 1)
        city_w = city_w.reshape(-1, city_w.shape[-2], city_w.shape[-1])
        # print(city_aqi.shape,city_conn.shape, city_w.shape)
        city_x, city_weight, city_conn = self.batchInput(
            city_aqi, city_w, city_conn)
        out = self.city_gnn(city_x, city_conn, city_weight)
        out = out.reshape(-1, 24, city_num, out.shape[-1])

        return out


class CityGNN(torch.nn.Module):
    def __init__(self, node_h, edge_h, gnn_h):
        super(CityGNN, self).__init__()
        self.node_mlp_1 = Seq(Lin(2 * node_h + edge_h, gnn_h),
                              ReLU(inplace=True))
        self.node_mlp_2 = Seq(Lin(node_h + gnn_h, gnn_h), ReLU(inplace=True))

    def forward(self, x, edge_index, edge_attr):
        # x: [N, F_x], where N is the number of nodes.
        # edge_index: [2, E] with max entry N - 1.
        # edge_attr: [E, F_e]
        row, col = edge_index
        out = torch.cat([x[row], x[col], edge_attr], dim=1)
        out = self.node_mlp_1(out)
        out = scatter_mean(out, col, dim=0, dim_size=x.size(0))
        out = torch.cat([x, out], dim=1)
        return self.node_mlp_2(out)


class CityModel(nn.Module):
    """Station graph"""
    def __init__(self, aqi_em, poi_em, wea_em, rnn_h, rnn_l, gnn_h):
        super(CityModel, self).__init__()
        self.rnn_h = rnn_h
        self.gnn_h = gnn_h
        self.rnn_l = rnn_l
        self.aqi_embed = Seq(Lin(1, aqi_em), ReLU())
        self.poi_embed = Seq(Lin(5, poi_em), ReLU())
        self.city_embed = Seq(Lin(gnn_h, wea_em), ReLU())
        self.wea_embed = Seq(Lin(5, wea_em), ReLU())
        self.sta_gnn = StaGNN(aqi_em + poi_em, 2, gnn_h, 2 * wea_em)
        self.encoder = RNNEncoder(input_size=gnn_h,
                                  hidden_size=rnn_h,
                                  num_layers=rnn_l)
        self.decoder_embed = Seq(Lin(1, aqi_em), ReLU())
        self.decoder = RNNDecoder(input_size=4 + aqi_em,
                                  hidden_size=rnn_h,
                                  num_layers=rnn_l)

    def batchInput(self, x, edge_w, edge_conn):
        sta_num = x.shape[1]
        x = x.reshape(-1, x.shape[-1])
        edge_w = edge_w.reshape(-1, edge_w.shape[-1])
        for i in range(edge_conn.size(0)):
            edge_conn[i, :] = torch.add(edge_conn[i, :], i * sta_num)
        edge_conn = edge_conn.transpose(0, 1)
        edge_conn = edge_conn.reshape(2, -1)
        return x, edge_w, edge_conn

    def forward(self, city_data, city_u, device):
        sta_aqi, sta_conn, sta_poi, sta_w, sta_wea, sta_for, _ = city_data
        sta_num = sta_aqi.shape[1]
        sta_x = sta_aqi.unsqueeze(dim=-1)
        sta_x = self.aqi_embed(sta_x)
        sta_poi = self.poi_embed(sta_poi)
        sta_poi = sta_poi.unsqueeze(dim=-2).repeat_interleave(24, dim=-2)
        sta_x = torch.cat([sta_x, sta_poi], dim=-1)
        sta_x = sta_x.transpose(1, 2)
        sta_x = sta_x.reshape(-1, sta_x.shape[-2], sta_x.shape[-1])

        sta_conn = sta_conn.transpose(1, 2).repeat(24, 1, 1)
        sta_w = sta_w.reshape(-1, sta_w.shape[-2], sta_w.shape[-1])
        # print(sta_x.shape,sta_conn.shape,sta_w.shape)
        sta_x, sta_weight, sta_conn = self.batchInput(sta_x, sta_w, sta_conn)
        city_u = self.city_embed(city_u)
        sta_wea = self.wea_embed(sta_wea)
        sta_u = torch.cat([city_u, sta_wea], dim=-1)
        sta_x = self.sta_gnn(sta_x, sta_conn, sta_weight, sta_u, sta_num)
        sta_x = sta_x.reshape(-1, 24, sta_num, sta_x.shape[-1]).transpose(1, 2)
        sta_x = sta_x.reshape(-1, 24, sta_x.shape[-1])

        h0 = torch.randn(self.rnn_l, sta_x.size(0), self.rnn_h).to(device)
        c0 = torch.randn(self.rnn_l, sta_x.size(0), self.rnn_h).to(device)
        h_x, c_x = self.encoder(sta_x, h0, c0)

        outputs = torch.zeros((sta_x.size(0), sta_for.size(1), 1)).to(device)
        aqi = sta_aqi[:, :, -1].reshape(-1, 1)
        sta_for = sta_for.repeat(sta_num, 1, 1)
        for i in range(sta_for.size(1)):
            aqi_em = self.decoder_embed(aqi)
            inputs = torch.cat((aqi_em, sta_for[:, i]), dim=-1)
            inputs = inputs.unsqueeze(dim=1)
            output, h_x, c_x = self.decoder(inputs, h_x, c_x)
            output = output.reshape(-1, 1)
            outputs[:, i] = output
            aqi = output
        outputs = outputs.reshape(-1, sta_num, sta_for.size(1))

        return outputs


class StaGNN(torch.nn.Module):
    def __init__(self, node_h, edge_h, gnn_h, u_h):
        super(StaGNN, self).__init__()
        self.node_mlp_1 = Seq(Lin(2 * node_h + edge_h, gnn_h),
                              ReLU(inplace=True))
        self.node_mlp_2 = Seq(Lin(node_h + gnn_h + u_h, gnn_h),
                              ReLU(inplace=True))

    def forward(self, x, edge_index, edge_attr, u, sta_num):
        # x: [N, F_x], where N is the number of nodes.
        # edge_index: [2, E] with max entry N - 1.
        # edge_attr: [E, F_e]
        u = u.reshape(-1, u.shape[-1])
        u = u.repeat(sta_num, 1)
        row, col = edge_index
        out = torch.cat([x[row], x[col], edge_attr], dim=1)
        out = self.node_mlp_1(out)
        out = scatter_mean(out, col, dim=0, dim_size=x.size(0))
        out = torch.cat([x, out, u], dim=1)
        return self.node_mlp_2(out)

## Train the model


In [ ]:
parser = argparse.ArgumentParser(description='Multi-city AQI forecasting')
parser.add_argument('--device', type=str, default='cuda', help='')
parser.add_argument('--run_times', type=int, default=5, help='')
parser.add_argument('--epoch', type=int, default=300, help='')
parser.add_argument('--batch_size', type=int, default=128, help='')
parser.add_argument('--city_num', type=int, default=10, help='')
parser.add_argument('--gnn_h', type=int, default=32, help='')
parser.add_argument('--rnn_h', type=int, default=64, help='')
parser.add_argument('--rnn_l', type=int, default=1, help='')
parser.add_argument('--aqi_em', type=int, default=16, help='')
parser.add_argument('--poi_em', type=int, default=8, help='poi embedding')
parser.add_argument('--wea_em', type=int, default=12, help='wea embedding')
parser.add_argument('--lr', type=float, default=0.001, help='lr')
parser.add_argument('--wd', type=float, default=0.001, help='weight decay')
parser.add_argument('--pred_step', type=int, default=12, help='step')
args = parser.parse_args()
print('this is arges',args)
device = args.device

In [19]:
#global_model = GlobalModel(args.aqi_em, args.rnn_h, args.rnn_l,
#                            args.gnn_h).to(device)
##jiaxing_model = CityModel(args.aqi_em, args.poi_em, args.wea_em,
#                          args.rnn_h, args.rnn_l, args.gnn_h).to(device)
#shanghai_model = CityModel(args.aqi_em, args.poi_em, args.wea_em,
#                            args.rnn_h, args.rnn_l, args.gnn_h).to(device)
#suzhou_model = CityModel(args.aqi_em, args.poi_em, args.wea_em, args.rnn_h,
#                          args.rnn_l, args.gnn_h).to(device)

global_model = GlobalModel(16, 64, 1,
                            32).to(device)
jiaxing_model = CityModel(16, 8, 12,
                          64, 1, 32).to(device)
shanghai_model = CityModel(16, 8, 12,
                          64, 1, 32).to(device)
suzhou_model = CityModel(16, 8, 12,
                          64, 1, 32).to(device)

city_model_num = sum(p.numel() for p in global_model.parameters()
                      if p.requires_grad)
print('city_model:', 'Trainable,', city_model_num)

shanghai_model_num = sum(p.numel() for p in shanghai_model.parameters()
                          if p.requires_grad)
print('shanghai_model_num:', 'Trainable,', shanghai_model_num)

criterion = nn.MSELoss()
params = list(global_model.parameters()) + list(jiaxing_model.parameters()) + \
    list(shanghai_model.parameters()) + list(suzhou_model.parameters())
optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=0.001)

val_loss_min = np.inf

city_model: Trainable, 28320
shanghai_model_num: Trainable, 51973
